In [6]:
from bs4 import BeautifulSoup
import requests
import csv

In [ ]:
url = "http://www.xssed.com/archive/page={}/"

In [4]:
mirrors = []
for i in range(1,1530):
    r = requests.get(url.format(i))
    soup = BeautifulSoup(r.content,"html5lib")

    for link in soup.find_all('a'):
        if "mirror" in link.get("href"):
            mirrors.append(link.get("href"))

In [ ]:
url = "http://www.xssed.com{}"
malicious_urls = []
mirror_urls = []

for m in mirrors:
    mirror = url.format(m)
    r = requests.get(mirror)
    soup = BeautifulSoup(r.content,"html5lib")
    malicious_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(4) > th")[0].text[5:])
    mirror_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(5) > th > a")[0].get("href"))

In [8]:
def write_file(filename,strings):
    with open(filename, "w") as file:
        for string in strings:
            file.write(string + "\n")

write_file("mirrors.txt",mirror_urls)
write_file("malicious_urls.txt",malicious_urls)

In [5]:
def file_to_list(filename):
    lines = []
    with open(filename,'r') as file:
        for line in file:
            cleaned_line = line.strip()
            lines.append(cleaned_line)
    return lines

In [15]:
import csv
import random

data = []
def generate_csv():
    malicious = file_to_list("malicious_urls.txt")
    benign = file_to_list("benign_urls.txt")
    for m in malicious:
        data.append([m,1])
    for b in benign:
        data.append([b,0])
    random.shuffle(data)

    with open("urls.csv",mode="w",newline="") as file:
        csv_writer = csv.writer(file)
        for row in data:
            print(row)
            csv_writer.writerow(row)
    file.close()

In [ ]:
generate_csv()

In [4]:
def read_csv(filename):
    with open(filename,mode="r") as file:
        csv_reader = csv.reader(file)
        data = []
        for row in csv_reader:
            data.append(row)
        return data

In [43]:
import re
import urllib

def duplicate(url):
    patterns = [r'<<', r'>>', r'%3C%3']
    total_occurrences = 0
    for pattern in patterns:
        occurrences = len(re.findall(pattern, url))
        total_occurrences += occurrences
    return total_occurrences

def special(url):
    substrings = ["“, “”>", "“>", "“/>"]
    total_occurrences = 0
    for substring in substrings:
        total_occurrences += url.count(substring)
    return total_occurrences

# def scripts(url):
#     pattern = re.compile("script", re.IGNORECASE)
#     matches = pattern.findall(url)
#     return len(matches)

def html_tags(url):
    pattern = r'<[^>]+>'
    html_tags = re.findall(pattern, url)
    return len(html_tags)

def cookie(url):
    pattern = r'document\.cookie'
    if re.search(pattern, url):
        return 1
    return 0

def redirections(url):
    pattern = r'window\.(location\.href|history\.back|navigate)'
    if re.search(pattern, url):
       return 1
    return 0

def extract_features(url):
    # attackers use url encoding to hide payloads / attack vectors
    # hence we decode it
    decoded_url = urllib.parse.unquote(url)

    return [len(decoded_url), 1 if decoded_url == url else 0 ,html_tags(decoded_url),duplicate(decoded_url),special(decoded_url),cookie(decoded_url),redirections(decoded_url)]

In [44]:
dataset = [["number_of_characters","url_encoding","html_tags","duplicate_characters","cookie_requests","redirections","target"]]

with open("urls.csv",mode="r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        new_row = extract_features(row[0])
        new_row.append(int(row[1]))
        dataset.append(new_row)
file.close()

with open("urls_features_data.csv",mode="w",newline="") as file:
    csv_writer = csv.writer(file)
    for row in dataset:
        csv_writer.writerow(row)

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv("urls_features_data.csv")
X = df.drop('target',axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9890582624146638
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7069
           1       1.00      0.97      0.98      3624

    accuracy                           0.99     10693
   macro avg       0.99      0.98      0.99     10693
weighted avg       0.99      0.99      0.99     10693



In [53]:
from sklearn.svm import SVC

clf = SVC(kernel='linear', C=1)  
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9890582624146638
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7069
           1       1.00      0.97      0.98      3624

    accuracy                           0.99     10693
   macro avg       0.99      0.98      0.99     10693
weighted avg       0.99      0.99      0.99     10693



In [54]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)  
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9890582624146638
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7069
           1       1.00      0.97      0.98      3624

    accuracy                           0.99     10693
   macro avg       0.99      0.98      0.99     10693
weighted avg       0.99      0.99      0.99     10693



In [57]:
line = "a = b"
line.split("var ")

['a = b']

In [9]:
import esprima

# sources
# document.URL
# document.documentURI
# document.URLUnencoded
# document.baseURI
# location.search
# document.cookie
# document.referrer

# sinks
# document.write()
# document.writeln()
# document.domain
# element.innerHTML
# element.outerHTML
# element.insertAdjacentHTML
# element.onevent
MEMBER_EXPRESSION = "MemberExpression"
DOCUMENT = "document"
LOCATION = "location"
PROPERTY = "property"
NAME = "name"
OBJECT = "object"
TYPE = "type"
IDENTIFIER = "Identifier"
TEMPLATE_LITERAL = "TemplateLiteral"
EXPRESSIONS = "expressions"
ARGUMENT = "argument"
# controlled = [{varname:[sources]}]
controlled = []
# {name, params_that_are_passed_to_a_sink}
functions = []
vulnerable_sinks = []

element_sinks = ["innerHTML","outerHTML","insertAdjacentHTML","onevent"]
document_sinks = ["domain"]
document_sources = ["URL","documentURI","URLUnencoded","baseURI","cookie","referrer"]
location_sources = ["search"]

def get_sources(expr):

    single_child_expressions = ["UpdateExpression",'AwaitExpression','UnaryExpression']
    double_child_expressions = ['BinaryExpression','LogicalExpression']

    if expr[TYPE] == MEMBER_EXPRESSION:
        if expr[OBJECT][NAME] == DOCUMENT and expr[PROPERTY][NAME] in document_sources:
            return [expr[PROPERTY][NAME]]
        if expr[OBJECT][NAME] == LOCATION and expr[PROPERTY][NAME] in location_sources:
            return [expr[PROPERTY][NAME]]
        
    elif expr[TYPE] == IDENTIFIER:
        return controlled[expr[NAME]]
    
    elif expr[TYPE] == TEMPLATE_LITERAL:
        sources = []
        for expr in expr[EXPRESSIONS]:
            sources.extend(get_sources(expr))
        return sources

    elif expr[TYPE] in single_child_expressions:
        return get_sources(expr[ARGUMENT])
    
    elif expr[TYPE] in double_child_expressions:
        return get_sources(expr["left"]).extend(get_sources(expr["right"]))
    
    elif expr[TYPE] == 'ConditionalExpression':
        return get_sources(expr["consequent"],expr["alternate"])

    # todo -> array_expression, object_expr, func_expr

    return []

def is_controlled(expr):
    return get_sources(expr) > 0

def is_sink(expr):

    if expr[TYPE] == MEMBER_EXPRESSION:
        if expr[PROPERTY][NAME] in element_sinks:
            return True
        elif expr[OBJECT][NAME] == DOCUMENT and expr[PROPERTY][NAME] in document_sinks:
            return True
    return False



def parsejs(code):
    return esprima.parseScript(code)

def tokenizejs(code):
    return esprima.tokenize(code)

def findSinks(expr):
    return []

def varDecl(declarations):
    sources = []
    for declaration in declarations:
        if declaration["id"] == "Identifer":
            if declaration["init"]:
                sources = findSources(declaration["init"])
                if len(sources) > 0:
                    controlled.append({declaration["id"]["name"]:sources})



def assignment(expr):
    if is_sink(expr["left"]) and is_controlled(expr["right"]):
        vulnerable_sinks.append({expr["left"]:get_sources(expr["right"])})
    elif is_controlled(expr["right"]):
        controlled.append( {expr["left"]:get_sources(expr["right"])} )



    



In [20]:
import esprima

def parsejs(code):
    return esprima.parseScript(code)

code = """ 
    const a = document.innerHTML + something
"""
code2 = """
    const [first,second] = something
"""

code3 = """
    document.innerHTML = location.search
    a = b
"""

code4 = """
function doSearchQuery(query) {
    document.getElementById('searchMessage').innerHTML = query;
}
"""

code5 = """ 
    document.innerHTML = "this"
    document.domain = 'some' 
    some.some.domain
"""

parsejs(code5)

{
    type: "Program",
    sourceType: "script",
    body: [
        {
            type: "ExpressionStatement",
            expression: {
                type: "AssignmentExpression",
                operator: "=",
                left: {
                    type: "MemberExpression",
                    computed: False,
                    object: {
                        type: "Identifier",
                        name: "document"
                    },
                    property: {
                        type: "Identifier",
                        name: "innerHTML"
                    }
                },
                right: {
                    type: "Literal",
                    value: "this",
                    raw: "\"this\""
                }
            }
        },
        {
            type: "ExpressionStatement",
            expression: {
                type: "AssignmentExpression",
                operator: "=",
                left: {
                    type: "